#### RNN Evaluation: Base vs Tuning Performance

In [2]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler

# Add path to the root folder
sys.path.append('../../../../../')
sys.path.append('../../../../features/prediction/')

from models.features.prediction.putils.formatter import create_sequences
from models.features.prediction.config.control import CONFIG
from models.features.prediction.config.path import BASE_DATASET_PATH
from models.features.prediction.manager import DataManager
from models.features.prediction.putils.observation import compute_rmse

In [3]:
# Load Dataset
selected_feature = "cpu_usage"
size_train, size_test = 1000, 250
dataset = DataManager.LoadDataset("../../../../../" + BASE_DATASET_PATH)[selected_feature]
scaler = MinMaxScaler(feature_range=(0, 1))
training_dataset = dataset[:size_train]
scaled_training_dataset = scaler.fit_transform(training_dataset.values.reshape(-1, 1))

In [4]:
# Define Performance Evaluation Function
def evaluate_model_performance(model, X_test, y_test, scaler):
    # Make predictions
    predictions = model.predict(X_test)

    # Inverse transform to get back to the original scale of the data
    y_test_original = scaler.inverse_transform(y_test)
    predictions_original = scaler.inverse_transform(predictions)

    # Calculate RMSE
    rmse = compute_rmse(y_test_original, predictions_original)
    return rmse

In [5]:
# Define Model
def create_rnn_2_layers_model(X, y, config):
    # RNN Model
    model = Sequential()
    model.add(
        SimpleRNN(
            config.get("neurons_l1", 50),
            activation=config.get("activation_function_l1", "relu"),
            input_shape=(X.shape[1], X.shape[2]),
            return_sequences=True,
        )
    )
    model.add(SimpleRNN(config.get("neurons_l2", 50), activation=config.get("activation_function_l2", "relu")))
    model.add(Dense(y.shape[1]))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=config.get("learning_rate", 0.001)
        ),
        loss="mse",
    )

    # Train the model
    model.fit(
        X,
        y,
        epochs=config.get("epochs", 1),
        verbose=config.get("verbose", 0),
        batch_size=config.get("batch_size", 32),
        validation_split=config.get("validation_split", 0.2),
    )
    return model

##### Base Version

In [6]:
base_config = {
    "n_past": 30,
    "epochs": 50,
    "neurons": 50,
    "batch_size": 32,
    "learning_rate": 0.001,
    "activation_function": "relu",
}

n_past = base_config["n_past"]
n_future = CONFIG["PREDICTION_STEPS"]
testing_dataset = dataset[size_train - n_past : size_train + size_test]
scaled_testing_dataset = scaler.transform(testing_dataset.values.reshape(-1, 1))

X_train, y_train = create_sequences(scaled_training_dataset, n_past, n_future)
model = create_rnn_2_layers_model(X_train, y_train, base_config)

# Evaluate the model performance
X_test, y_test = create_sequences(scaled_testing_dataset, n_past, n_future)
rmse = evaluate_model_performance(model, X_test, y_test, scaler)
print("Model RMSE:", rmse)

8/8 [==============================] - 0s 5ms/step
Model RMSE: 0.031888830843966834


##### Tuned Version (L2)

In [7]:
# Best parameters:
#               - n_past=36
#               - epochs=74
#               - batch_size=35
#               - learning_rate=0.045884
#               - neurons_l1=49
#               - neurons_l2=55
#               - activation_function_l1=sigmoid
#               - activation_function_l2=sigmoid
l2_config = {
    "n_past": 36,
    "epochs": 74,
    "batch_size": 35,
    "learning_rate": 0.045884,
    "neurons_l1": 49,
    "neurons_l2": 55,
    "activation_function_l1": "sigmoid",
    "activation_function_l2": "sigmoid",
}

n_past = l2_config["n_past"]
n_future = CONFIG["PREDICTION_STEPS"]
testing_dataset = dataset[size_train - n_past : size_train + size_test]
scaled_testing_dataset = scaler.transform(testing_dataset.values.reshape(-1, 1))

X_train, y_train = create_sequences(scaled_training_dataset, n_past, n_future)
model = create_rnn_2_layers_model(X_train, y_train, l2_config)

# Evaluate the model performance
X_test, y_test = create_sequences(scaled_testing_dataset, n_past, n_future)
rmse = evaluate_model_performance(model, X_test, y_test, scaler)
print("Model RMSE:", rmse)

8/8 [==============================] - 0s 9ms/step
Model RMSE: 0.029282276628641527
